In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import csv
import json
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize

# Download NLTK resources if needed
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

# Define science/health/nutrition related terms
TOPIC_KEYWORDS = {
    'science': [
        'science', 'research', 'experiment', 'theory', 'scientific', 'scientist', 'study', 
        'laboratory', 'evidence', 'hypothesis', 'analysis', 'discovery', 'biology', 'physics', 
        'chemistry', 'astronomy', 'geology'],
    'health': [
        'health', 'medical', 'medicine', 'disease', 'doctor', 'patient', 'treatment', 'cure', 
        'symptoms', 'diagnosis', 'therapy', 'wellness', 'illness', 'healthcare', 'hospital', 
        'clinic', 'recovery', 'prevention', 'condition', 'immune', 'surgery', 'mental health',
        'physical', 'wellbeing', 'disease', 'cancer', 'heart', 'brain', 'lungs'
    ],
    'nutrition': [
        'nutrition', 'diet', 'food', 'healthy eating', 'nutrient', 'vitamin', 'mineral', 
        'protein', 'carbohydrate', 'fat', 'calorie', 'metabolism', 'digestion', 'meal', 
        'vegetable', 'fruit', 'meat', 'dairy', 'weight', 'obesity', 'sugar', 'organic', 
        'supplement', 'fiber', 'hydration', 'fasting', 'macronutrient'
    ]
}

# Flatten the keywords list for easier checking
ALL_KEYWORDS = set()
for category_keywords in TOPIC_KEYWORDS.values():
    ALL_KEYWORDS.update(category_keywords)

def clean_text(text):
    """Clean and normalize text"""
    if not text:
        return ""
    # Replace newlines and multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters except punctuation
    text = re.sub(r'[^\w\s.,!?;:\'\"-]', '', text)
    return text.strip()

def count_words(text):
    """Count words in text"""
    if not text:
        return 0
    words = word_tokenize(text)
    return len(words)

def is_relevant_topic(text):
    """
    Check if the text is relevant to science, health, or nutrition
    using keyword matching
    """
    if not text:
        return False
    
    # Clean and lowercase the text
    clean_text_lower = text.lower()
    
    # Direct keyword matching
    for keyword in ALL_KEYWORDS:
        if keyword.lower() in clean_text_lower:
            return True
    
    return False

def process_csv_file(input_file, output_jsonl, output_csv, min_words=100, max_words=120, 
                   max_entries=2000, encoding='utf-8'):
    """
    Process the CSV file, filter entries based on criteria, and save output
    """
    print(f"Reading file: {input_file}")
    try:
        # Read the CSV file
        df = pd.read_csv(input_file, encoding=encoding)
        print(f"Successfully loaded {len(df)} entries from {input_file}")
        print(f"Column headers: {list(df.columns)}")
    except Exception as e:
        print(f"Error reading file {input_file}: {e}")
        return []
    
    # Filter and process entries
    valid_entries = []
    
    with open(output_jsonl, 'w', encoding='utf-8') as out_f:
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing entries"):
            try:
                # Get the text content and clean it
                content = str(row.get('content', ''))
                text = clean_text(content)
                
                # Skip if text is empty
                if not text:
                    continue
                
                # Count words (we'll calculate ourselves rather than using tokens)
                word_count = count_words(text)
                
                # Get the input query
                input_text = str(row.get('input', ''))
                input_text = clean_text(input_text)
                
                # Check if the text is relevant to our topics
                is_relevant = is_relevant_topic(text)
                
                # Check if the entry meets our criteria
                if min_words <= word_count <= max_words and is_relevant:
                    # Create a new entry with the fields we want to keep
                    filtered_entry = {
                        'id': row.get('uuid', f"entry_{idx}"),
                        'content': text,
                        'word_count': word_count,
                        'input': input_text,
                        'source': row.get('repo_name', '')
                    }
                    
                    # Write directly to output JSONL file
                    out_f.write(json.dumps(filtered_entry) + '\n')
                    
                    # Append to our valid entries list
                    valid_entries.append(filtered_entry)
                    
                    # Print progress periodically
                    if len(valid_entries) % 10 == 0:
                        print(f"Found {len(valid_entries)} valid entries so far")
                    
                    # Stop if we've reached the desired number of entries
                    if len(valid_entries) >= max_entries:
                        print(f"Reached {max_entries} entries, stopping.")
                        break
            except Exception as e:
                print(f"Error processing entry {idx}: {e}")
                continue
    
    print(f"Found and saved {len(valid_entries)} entries matching criteria to {output_jsonl}")
    
    # Convert to DataFrame and save as CSV
    if valid_entries:
        valid_df = pd.DataFrame(valid_entries)
        valid_df = valid_df[['content', 'word_count']]
        valid_df.to_csv(output_csv, index=False, quoting=csv.QUOTE_ALL)
        print(f"Converted entries saved to {output_csv}")
        
        # Print some statistics
        word_counts = [entry['word_count'] for entry in valid_entries]
        if word_counts:
            avg_word_count = sum(word_counts) / len(word_counts)
            print(f"Average word count: {avg_word_count:.2f}")
            print(f"Min word count: {min(word_counts)}")
            print(f"Max word count: {max(word_counts)}")
            
            # Word count statistics
            word_count_df = pd.DataFrame({'word_count': word_counts})
            stats = word_count_df.describe()
            print("Word Count Statistics:")
            print(stats)
            
            # Plot histogram
            plt.figure(figsize=(10, 6))
            plt.hist(word_counts, bins=20, alpha=0.7)
            plt.title('Word Count Distribution')
            plt.xlabel('Word Count')
            plt.ylabel('Frequency')
            plt.grid(alpha=0.3)
            plt.savefig('word_count_distribution.png')
            plt.close()
            print("Word count distribution plot saved to 'word_count_distribution.png'")
            
            # Source distribution if available
            if 'source' in valid_df.columns:
                source_counts = valid_df['source'].value_counts()
                print("\nSource Distribution:")
                print(source_counts)
                
                # Plot source distribution
                plt.figure(figsize=(10, 6))
                source_counts.plot(kind='bar')
                plt.title('Source Distribution')
                plt.xlabel('Source')
                plt.ylabel('Count')
                plt.grid(axis='y', alpha=0.3)
                plt.savefig('source_distribution.png')
                plt.close()
                print("Source distribution plot saved to 'source_distribution.png'")
    
    return valid_entries

if __name__ == "__main__":
    # Configuration
    input_file = "train.csv"  # CSV input file (change to your actual filename)
    output_jsonl = "filtered_train.jsonl"  # Output JSONL file
    output_csv = "filtered_train.csv"  # Output CSV file
    min_words = 100  # Minimum word count
    max_words = 120  # Maximum word count
    max_entries = 2000  # Maximum number of entries to collect
    
    # Run the processor
    filtered_data = process_csv_file(
        input_file=input_file,
        output_jsonl=output_jsonl,
        output_csv=output_csv,
        min_words=min_words,
        max_words=max_words,
        max_entries=max_entries
    )
    
    print(f"Processing complete. {len(filtered_data)} entries filtered and saved.")

Reading file: train.csv
Successfully loaded 110000 entries from train.csv
Column headers: ['uuid', 'content', 'content_tokens_len', 'input', 'prompt_tokens_len', 'reasoning_content', 'reasoning_content_tokens_len', 'repo_name', 'score']


Processing entries:   1%|          | 1233/110000 [00:03<04:22, 414.65it/s]

Found 10 valid entries so far


Processing entries:   2%|▏         | 2045/110000 [00:05<04:16, 421.31it/s]

Found 20 valid entries so far


...

Found 1190 valid entries so far


Processing entries: 100%|█████████▉| 109621/110000 [04:07<00:01, 314.58it/s]

Found 1200 valid entries so far


Processing entries: 100%|██████████| 110000/110000 [04:08<00:00, 441.83it/s]


Found and saved 1201 entries matching criteria to filtered_train.jsonl
Converted entries saved to filtered_train.csv
Average word count: 110.00
Min word count: 100
Max word count: 120
Word Count Statistics:
        word_count
count  1201.000000
mean    110.000833
std       5.971111
min     100.000000
25%     105.000000
50%     110.000000
75%     115.000000
max     120.000000
Word count distribution plot saved to 'word_count_distribution.png'
Processing complete. 1201 entries filtered and saved.
